#!conda install -y cudatoolkit
#!pip install bitsandbytes
!pip install transformers==4.33.2
!pip install datasets
!pip install evaluate
!pip install peft
!pip install accelerate -U
!pip install transformers[torch]
!pip install -r requirements.txt

In [2]:
%pip install -q -U torch==2.0.1 bitsandbytes==0.40.2
%pip install -q -U transformers==4.31.0 peft==0.4.0 accelerate==0.21.0
%pip install -q -U datasets py7zr einops tensorboardX
!pip install evaluate
# Add installed cuda runtime to path for bitsandbytes
import os
import nvidia

cuda_install_dir = '/'.join(nvidia.__file__.split('/')[:-1]) + '/cuda_runtime/lib/'
os.environ['LD_LIBRARY_PATH'] =  cuda_install_dir


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any

In [3]:
from collections import defaultdict
import copy
import json
import os
import sys
#sys.path.append("/home/sagemaker-user/bitsandbytes")
#os.environ['LD_LIBRARY_PATH'] = '/opt/conda/lib/'
from os.path import exists, join, isdir
from dataclasses import dataclass, field
import sys
from typing import Optional, Dict, Sequence
import numpy as np
from tqdm import tqdm
import logging
#import bitsandbytes as bnb
import pandas as pd

import torch
torch.cuda.init()
print(torch.cuda.is_available())
import os
#os.environ['TRANSFORMERS_CACHE'] = '/project/SDS/research/christ_research/Llama 2/llama2-7b/cache'
import transformers
from torch.nn.utils.rnn import pad_sequence
import argparse
# from transformers import (
#     AutoTokenizer,
#     AutoModelForCausalLM,
#     set_seed,
#     Seq2SeqTrainer,
# #    BitsAndBytesConfig,
#     LlamaTokenizer

# )
from datasets import load_dataset, Dataset
import evaluate

# from peft import (
#     prepare_model_for_kbit_training,
#     LoraConfig,
#     get_peft_model,
#     PeftModel,
#     TaskType
# )
#from peft.tuners.lora import LoraLayer
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
#from dotenv import load_dotenv
import os
torch.backends.cuda.matmul.allow_tf32 = True

# Load the environmental variables from the .env file
#load_dotenv()

#token = os.getenv('text_classifier_token')

#from huggingface_hub import login
#login(token = token)

from datasets import load_dataset, DatasetDict

True


In [4]:
import random

In [72]:
df = pd.read_csv('app/feedback.csv')
pos = df[df['label']==1]
neg = df[df['label']==0]
neg_len = len(neg)
pos = pos.sample(n = neg_len, replace = False)
df2 = pd.concat([pos, neg])
df2 = df2.sample(frac=1)
df2.reset_index(drop=True, inplace=True)
df2.to_csv('balanced.csv', index= False)
df2

,label,text
0,0,Question: Ossory has $33.00. He buys 2124323 p...
1,1,Question: A box of cheese crackers was on sale...
2,1,Question: 100 chocolate chips make 5 cookies. ...
3,1,Question: The 76ers are giving away two free t...
4,0,Question: Fibonacci Sequence: Solution: \ndef ...
...,...,...
385,1,Question: A pizzeria sells 5 pizzas and an ent...
386,1,Question: Lydia is Lego-Leotards. She sells on...
387,1,Question: A 100-page book was translated from ...
388,0,Question: Mark was baking pies for a baking co...


In [13]:
# Load the dataset
dataset = load_dataset('csv', data_files="data/appropriateness.csv")
#dataset = load_dataset('csv', data_files="balanced.csv")

# Get the total number of examples in the dataset
# total_examples = len(dataset['train'])

# # Calculate the sizes of the training, test, and validation sets
# train_size = int(0.8 * total_examples)
# test_size = int(0.1 * total_examples)
# valid_size = total_examples - train_size - test_size

# # Manually split the dataset into training, test, and validation sets
# train_dataset = dataset['train'].shuffle(seed=42).select(range(train_size))
# test_dataset = dataset['train'].shuffle(seed=42).select(range(train_size, train_size + test_size))
# valid_dataset = dataset['train'].shuffle(seed=42).select(range(train_size + test_size, total_examples))

dataset_train_valid_test = dataset['train'].train_test_split(test_size = .2, seed = 42)
dataset_valid_test = dataset_train_valid_test['test'].train_test_split(test_size = .5, seed = 42)
# Create a DatasetDict to hold the splits
train_test_valid_dataset = DatasetDict({
    'train': dataset_train_valid_test['train'],
    'test': dataset_valid_test['train'],
    'valid': dataset_valid_test['test']
})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
train_test_valid_dataset['valid'][2]

{'Unnamed: 0': 1299,
 'label': 1,
 'text': 'Question: Thor is a God. He is 500 times stronger than the strongest human. The strongest human can lift 500 pounds. How many pounds can Thor lift?\nSolution:\ndef solution():\n    #The strongest human can lift 500 pounds\n    strongest_human = 500\n    #Thor is 500 times stronger than the strongest human\n    thor = 500 * strongest_human\n    return thor\nAnswer: 250000'}

In [32]:
#Set up tokenizer
from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", use_auth_token=True)
# tokenizer.add_special_tokens({"pad_token":"[PAD]"})

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
#tokenizer.add_special_tokens({"pad_token":"[PAD]"})

#Preprocess and collate data
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train_test_valid_dataset = train_test_valid_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1188 [00:00<?, ? examples/s]

Map:   0%|          | 0/149 [00:00<?, ? examples/s]

Map:   0%|          | 0/149 [00:00<?, ? examples/s]

In [33]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#Prepare evaluation function
import evaluate
accuracy = evaluate.load("accuracy")

import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# roc_auc = evaluate.load("roc_auc")

# import numpy as np
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return roc_auc.compute(prediction_scores=predictions, references=labels)

#Training
id2label = {0: "NOT APPROPRIATE", 1: "APPROPRIATE"}
label2id = {"NOT APPROPRIATE": 0, "APPROPRIATE": 1}

In [34]:
#Import model
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# model = AutoModelForSequenceClassification.from_pretrained(
#     "meta-llama/Llama-2-7b-hf", num_labels=2, id2label=id2label, label2id=label2id,
#         use_auth_token=True,  
#       # max_memory=max_memory,
#       # torch_dtype=torch.bfloat16, 
#       device_map = 'auto',
#         quantization_config=BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_compute_dtype=torch.bfloat16,
#             bnb_4bit_use_double_quant=True,
#             bnb_4bit_quant_type='nf4'))

# model = AutoModelForSequenceClassification.from_pretrained(
#     "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id,
#         use_auth_token=True,  
#       # max_memory=max_memory,
#       # torch_dtype=torch.bfloat16, 
#         quantization_config=BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_compute_dtype=torch.bfloat16,
#             bnb_4bit_use_double_quant=True,
#             bnb_4bit_quant_type='nf4'))

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
#NEWLY ADDED
# model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)

# peft_config = LoraConfig(
#     task_type=TaskType.SEQ_CLS,
#     r=16,
#     lora_alpha=32,
#     lora_dropout=0.1,
# )      

# model = get_peft_model(model, peft_config)

#END NEWLY ADDED
# training_args = TrainingArguments(
#     output_dir="text_classifier_llama",
#     learning_rate=2e-5,
#     per_device_train_batch_size=1,
#   # auto_find_batch_size = True,
#     num_train_epochs=8,
#     weight_decay=0.01,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True, 
#     bf16=True)

training_args = TrainingArguments(
    output_dir="appropriateness_classifier",
    learning_rate=.0001,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_test_valid_dataset["train"],
    eval_dataset=tokenized_train_test_valid_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
print(tokenized_train_test_valid_dataset["test"])
test_input_data = tokenized_train_test_valid_dataset["test"]["input_ids"]

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.638873,0.664430
2,No log,0.669346,0.664430


KeyboardInterrupt: 

In [30]:
# predictions = trainer.predict(test_dataset=test_input_data)
# print(predictions)
trainer.evaluate(eval_dataset = tokenized_train_test_valid_dataset["test"])

{'eval_loss': 0.540840208530426, 'eval_roc_auc': 0.5}

In [31]:
trainer.predict(test_dataset = tokenized_train_test_valid_dataset["test"])

PredictionOutput(predictions=array([[-1.3659638 , -0.3504559 ],
       [-1.3659235 , -0.35044563],
       [-1.3660057 , -0.35047084],
       [-1.3659629 , -0.35044515],
       [-1.3661035 , -0.35057852],
       [-1.3659449 , -0.35040134],
       [-1.3660743 , -0.35055506],
       [-1.3660606 , -0.35057616],
       [-1.3660786 , -0.35052443],
       [-1.3660532 , -0.35050067],
       [-1.3659946 , -0.35050136],
       [-1.3659312 , -0.3504006 ],
       [-1.3660996 , -0.35056302],
       [-1.3659462 , -0.35042378],
       [-1.3661294 , -0.35062513],
       [-1.3660597 , -0.35052636],
       [-1.3660964 , -0.3506304 ],
       [-1.365994  , -0.35047683],
       [-1.3661128 , -0.35062638],
       [-1.366017  , -0.350502  ],
       [-1.3659413 , -0.35041237],
       [-1.3660183 , -0.35052314],
       [-1.3657619 , -0.35029495],
       [-1.3659871 , -0.3505102 ],
       [-1.3658106 , -0.35033143],
       [-1.3660417 , -0.35052323],
       [-1.3660954 , -0.35052058],
       [-1.3660591 , -0.35

# Testing on IMDB Dataset

In [16]:
from datasets import load_dataset

imdb = load_dataset("imdb")

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_imdb = imdb.map(preprocess_function, batched=True)
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
import evaluate

accuracy = evaluate.load("accuracy")

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [17]:
import sys
#sys.path.append("/home/sagemaker-user/bitsandbytes")
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import bitsandbytes

In [20]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    output_dir="text_classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    evaluation_strategy="steps",  # Change evaluation strategy to "steps"
    eval_steps=100,  # Set the evaluation frequency to every 100 steps
    save_strategy="steps",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb['train'],
    eval_dataset=tokenized_imdb['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
100,No log,0.302497,0.876960
200,No log,0.250724,0.900440
300,No log,0.263815,0.904360


KeyboardInterrupt: 

In [6]:
conda list | grep cudatoolkit

cudatoolkit               11.8.0              h4ba93d1_12    conda-forge

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#11.8.0